In [1]:
import numpy as np
import pandas as pd

In [2]:
events = pd.read_excel("Event.xlsx")

In [3]:
events

,Events,Type,Unnamed: 2
0,Introducing The Management 2.0 Hackathon,Management.Hackathons,NaN
1,Join us live online on September 10th at 9:30 ...,Management.Webinars,NaN
2,Learn and practise your delegation skills with...,Management.Webinars,NaN
3,Upcoming Online Seminar: The Future of Managem...,Management.Webinars,NaN
4,World's biggest coding event - TechGig Code Gl...,Coding.Hackathons,NaN
...,...,...,...
347,Join the C Developer Conference happening Tomo...,C.Seminars,NaN
348,Seminar - Objective C and Its uses,C.Seminars,NaN
349,To C or Not to C. A deep dive conference aout ...,C.Seminars,NaN
350,C is everywhere but we just don’t see it. Join...,C.Seminars,NaN


In [4]:
target_names = events.Type.unique().tolist()

In [5]:
target_names

['Management.Hackathons',
 'Management.Webinars',
 'Coding.Hackathons',
 'Other.Hackathons',
 'Coding.Webinars',
 'Coding.Seminars',
 'Security.Webinars',
 'Security.Seminars',
 'Other.Certifications',
 'Security.Certifications',
 'Other.Competitions',
 'Management.Trainings',
 'Other.Webinar',
 'Higher_Education.Trainings',
 'Higher_Education.Internships',
 'Higher_Education.Webinars',
 'Higher_Education.Expos',
 'Higher_Education.Seminars',
 'Management.Expos',
 'Coding.Fests',
 'Coding.Internships',
 'Coding.Jobs',
 'Coding.Expos',
 'Coding.Certifications',
 'Coding.Talks',
 'Coding.Competitions',
 'Artificial_Intelligence.Hackathons',
 'Artificial_Intelligence.Webinars',
 'Artificial_Intelligence.Fests',
 'Artificial_Intelligence.Seminars',
 'Artificial_Intelligence.Expos',
 'Artificial_Intelligence.Certifications',
 'Artificial_Intelligence.Trainings',
 'Artificial_Intelligence.Courses',
 'Artificial_Intelligence.Jobs',
 'Artificial_Intelligence.Internships',
 'Artificial_Intellig

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(events.Events.str.lower().values, events.Type.values, test_size=0.1, shuffle = True, random_state=42)

In [7]:
# Extracting features from text files
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(316, 800)

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(316, 800)

In [16]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [18]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf = text_clf.fit(X_train, y_train)

In [19]:
import numpy as np
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.0

In [20]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=20, random_state=42))])

text_clf_svm = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)
for i in range(len(X_test)):
    if i%3 == 0:
        print(X_test[i] + " : "+ predicted_svm[i] + " : " + y_test[i])
np.mean(predicted_svm == y_test)

part time position as a mobile application executive : Finance.Internships : Mobile_Applications.Internships
job alert! join xl dynamics' elite team of financial analysts - interview @ home : Finance.Jobs : Finance.Jobs
how to program efficiently? join us in a live from your home : C.Webinars : Coding.Webinars
ai chat bot developer required! : Coding.Jobs : Artificial_Intelligence.Jobs
don’t waste your vacations. use it wisely by joining our bootcamp on mobile application managing ( both android and ios ) : Mobile_Applications.Trainings : Mobile_Applications.Trainings
one week away! participate in developer week: one of the world’s largest expo and conferences for developers : Python.Expos : Coding.Expos
harness the power of c++ with our playlist of talks : Mobile_Applications.Courses : Cpp.Talks
artificial intelligence hackathon by codebay. two days left! : Mobile_Applications.Hackathons : Artificial_Intelligence.Hackathons
the modern c++ course by udacity is out now! sign up today fo

0.19444444444444445

In [11]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

In [21]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)

C:\Users\shiva\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [22]:
gs_clf.best_score_
gs_clf.best_params_

{'clf__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}

In [23]:
from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}

gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)


gs_clf_svm.best_score_
gs_clf_svm.best_params_

C:\Users\shiva\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
C:\Users\shiva\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


{'clf-svm__alpha': 0.001, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}

In [24]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())])

In [ ]:
import nltk
nltk.download()

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect), ('tfidf', TfidfTransformer()), 
                             ('mnb', MultinomialNB(fit_prior=False))])

text_mnb_stemmed = text_mnb_stemmed.fit(X_train, y_train)

predicted_mnb_stemmed = text_mnb_stemmed.predict(X_test)

np.mean(predicted_mnb_stemmed == y_test)

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
